In [1]:
%pylab inline
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_hub as hub
tfd = tfp.distributions
tfb = tfp.bijectors
import pandas
from astropy.table import Table

Populating the interactive namespace from numpy and matplotlib


In [2]:
batch_size = 1024

# This  placeholder will hold the data, here it's 2d data
x = tf.placeholder(tf.float32, shape=(batch_size,2))

lr = tf.placeholder_with_default(0.001, shape=())

chain = [ tfb.RealNVP(1, shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[128,128],
                                                                      activation=tf.nn.leaky_relu)),
          tfb.Permute([1,0]),
          tfb.RealNVP(1, shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[128,128],
                                                                      activation=tf.nn.leaky_relu)),
          tfb.Permute([1,0]),
          tfb.RealNVP(1, shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[128,128],
                                                                        activation=tf.nn.leaky_relu)),
          tfb.Permute([1,0]),
          tfb.RealNVP(1, shift_and_log_scale_fn=tfb.real_nvp_default_template(hidden_layers=[128,128],
                                                                        activation=tf.nn.leaky_relu)),
        ]
# This defines the bijective mapping of the flow
bij = tfb.Chain(chain)

# This is the prior for a 2d space
prior = tfd.MultivariateNormalDiag(loc=tf.zeros(2), scale_identity_multiplier=1.0)

# That'st the model p_\theta of the target density, obtained by mapping the prior through the bijection
distribution = tfd.TransformedDistribution(prior, bijector=bij)

# This is the log probability of the data under the model
log_prob = distribution.log_prob(x)

W0828 18:32:45.551781 4598076864 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow_probability/python/bijectors/real_nvp.py:293: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0828 18:32:45.556859 4598076864 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [3]:
# The loss function for optimization is the negative log likelihood of the data under the model
loss = - tf.reduce_mean(log_prob)

In [4]:
optimizer = tf.train.AdamOptimizer(learning_rate=lr)
opt_op = optimizer.minimize(loss)

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [6]:
losses = []

In [7]:
df = pandas.read_feather('cmd1e6.dat')

In [8]:
data = df[['G-J', 'G_mag']].to_numpy()

In [9]:
for i in range(5000):
    x_batch = data[np.random.randint(0, len(data), batch_size)]
    _, l = sess.run([opt_op, loss], feed_dict={x:x_batch, lr:0.001})
    losses.append(l)
    if i %100 ==0:
        print(i, l)

0 118.57424
100 5.266345


KeyboardInterrupt: 

In [10]:
# Draw 1000 points from the learned distribution
fake = sess.run(distribution.sample(1000))

In [11]:
x = np.linspace(-1, 6, 100)
y = np.linspace(4, 18, 100)
xx, yy = np.meshgrid(x, y)
p = distribution.log_prob(np.array([xx.flatten(), yy.flatten()]).T)

In [12]:
prob = sess.run(p)

In [13]:
import matplotlib.pyplot as plt

In [14]:
print(x.shape, y.shape, prob.reshape(100, 100).shape)

(100,) (100,) (100, 100)


In [ ]:
plt.imshow(prob.reshape(100, 100))

In [ ]:
plt.pcolormesh(x, y, prob.reshape(100,100))